<a href="https://colab.research.google.com/github/BryanCruz/Netflickerson/blob/master/Netflickerson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projeto 2 - Parte B - Netflickerson**

Bryan Bialokur da Cruz - 11052316

Filipi de Carvalho Brabo - 11040016

### **Chatbot com Python e NLTK para pesquisar filmes e séries da Netflix**

O objetivo do projeto é criar um Chatbot que, a partir de uma sentença escrita pelo usuário, identifique alguns filmes e séries da Netflix que tenham relação com essa sentença. Além disso, o Chatbot também irá responder saudações e despedidas do usuário.

Como a base de dados utilizada está em inglês, fizemos o Chatbot responder apenas mensagens em inglês.

### **01. Importação das bibliotecas**

In [ ]:
import re
import nltk
import pandas as pd
import numpy as np
import random
import string

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
!pip install spacy --upgrade > /dev/null

In [ ]:
import spacy
spacy.__version__
#pip install spacy==2.2.3

'2.3.4'

In [ ]:
nltk.__version__

'3.2.5'

In [ ]:
!python3 -m spacy download en > /dev/null

### **02. Carregamento e pré-processamento da base de dados**

**Carregamento da base de dados**

Base de dados conteúdo informações sobre filmes e séries da Netflix, tais como título, diretor, elenco, descrição, etc.

In [ ]:
data = pd.read_csv('https://www.dropbox.com/s/gp7tzld5vq38hvi/netflix_titles.csv?dl=1')

In [ ]:
data = data.fillna("")
data.sample(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
3574,404067,Movie,The Craft,Andrew Fleming,"Robin Tunney, Fairuza Balk, Neve Campbell, Rachel True, Skeet Ulrich, Christine Taylor, Breckin Meyer, Nathaniel Marston, Cliff De Young, Assumpta Serna",United States,"January 1, 2020",1996,R,101 min,"Horror Movies, Thrillers","At a Catholic school, a new girl falls in with a clique of teen witches who wield their powers against anyone who crosses them, even their parents."
10,80169755,Movie,Joaquín Reyes: Una y no más,José Miguel Contreras,Joaquín Reyes,,"September 8, 2017",2017,TV-MA,78 min,Stand-Up Comedy,"Comedian and celebrity impersonator Joaquín Reyes decides to be his zesty self for a night of stories about buses, bathroom habits, royalty and more."
884,80187618,Movie,Manje Bistre,Baljit Singh Deo,"Gippy Grewal, Sonam Bajwa, Karamjit Anmol, Jaggi Singh, B.N. Sharma, Hobby Dhaliwal, Rana Ranbir, Sardar Sohi, Anita Devgan, Harby Sangha",India,"October 1, 2018",2017,TV-PG,136 min,"Comedies, Dramas, International Movies","While prepping for a family wedding, a young man falls for his sister’s friend, who has already been promised to someone else."


In [ ]:
print(data.columns)
print(data.dtypes)

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')
show_id          int64
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object


In [ ]:
interest_columns = ["type", "title", "director", "cast", "country", "release_year", "listed_in", "description"]

In [ ]:
data["words"] = data[interest_columns].astype("string").agg(' '.join, axis=1)

In [ ]:
data["words"].sample(5)

5268                                                                                                                                                                                    TV Show What in the World Happened?  Eun-mi Go, Jaehyo, Eun-Hyung Jo, Ha-yeon Kim, Shin Yi, Kwang-sun Park  2015 International TV Shows, Korean TV Shows, Romantic TV Shows A happy secret romance between a 40-year-old woman and a 25-year-old man is complicated after a greeting card grants her wish to be young again.
4434                                                                                        Movie For the Love of Spock Adam Nimoy Leonard Nimoy, William Shatner, George Takei, Nichelle Nichols, Chris Pine, Zachary Quinto, Simon Pegg, Zoe Saldana, Jim Parsons, Jason Alexander, Neil deGrasse Tyson, J.J. Abrams Canada, United States 2016 Documentaries The son of actor Leonard Nimoy directs this moving look at his father's portrayal of Spock, the iconic "Star Trek" character he played for 50 years

**Pré-processamento da base de dados**

In [ ]:
lista_sentencas = data["words"].str.lower().tolist()
lista_sentencas[0]

'movie norm of the north: king sized adventure richard finn, tim maltby alan marriott, andrew toth, brian dobson, cole howard, jennifer cameron, jonathan holmes, lee tockar, lisa durupt, maya kay, michael dobson united states, india, south korea, china 2019 children & family movies, comedies before planning an awesome wedding for his grandfather, a polar bear king must take back a stolen artifact from an evil archaeologist first.'

In [ ]:
len(lista_sentencas)

6234

In [ ]:
lista_sentencas[1]

'movie jandino: whatever it takes  jandino asporaat united kingdom 2016 stand-up comedy jandino asporaat riffs on the challenges of raising kids and serenades the audience with a rousing rendition of "sex on fire" in his comedy show.'

In [ ]:
pln = spacy.load('en')
pln

In [ ]:
stop_words = spacy.lang.en.stop_words.STOP_WORDS

In [ ]:
print(stop_words)

{'get', 'once', 'anyhow', 'herself', 'while', 'less', 'what', 'ca', 'show', 'regarding', 'its', 'we', 'no', 'nowhere', 'own', 'forty', 'well', 'somewhere', 'each', 'full', 'twelve', 'whenever', 'when', "'re", 'either', 'anyone', 'does', 'over', 'at', 'some', 'into', 'sometime', 'least', 'perhaps', 'thence', 'for', 'front', 'someone', 'by', 'one', 'where', '‘ve', 'everyone', 'under', 'whither', 'but', 'it', 'almost', 'neither', 'n’t', 'be', 'noone', 'hundred', 'most', "'m", 'namely', 'out', 'mine', 'take', 'behind', 'first', 'through', 'any', 'seem', '‘ll', 'an', 'thereby', 'per', 'himself', 'other', 'between', 'can', 'eight', 'further', 'anything', 'he', 'often', 'anyway', 'even', 'off', 'ourselves', 'serious', 'them', 'up', 'ever', 'hereby', 'were', "'ll", 'above', 'throughout', 'rather', 'two', 'elsewhere', 'various', 'others', 'again', '‘s', 'yours', 'although', 'yourselves', 'being', 'enough', 'hence', 'could', 'go', 'on', 'am', 'me', 'top', 'than', "'s", 'fifteen', 'her', 'thru', 

In [ ]:
type(stop_words)

set

In [ ]:
len(stop_words)

326

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def preprocessamento(texto):
  # espacos em branco
  texto = re.sub(r" +", ' ', texto)

  documento = pln(texto)
  lista = []
  for token in documento:
    lista.append(token.lemma_)

  lista = [palavra for palavra in lista if palavra not in stop_words and palavra not in string.punctuation]
  lista = ' '.join([str(elemento) for elemento in lista])

  return lista

In [ ]:
texto_teste = lista_sentencas[0]
texto_teste

'movie norm of the north: king sized adventure richard finn, tim maltby alan marriott, andrew toth, brian dobson, cole howard, jennifer cameron, jonathan holmes, lee tockar, lisa durupt, maya kay, michael dobson united states, india, south korea, china 2019 children & family movies, comedies before planning an awesome wedding for his grandfather, a polar bear king must take back a stolen artifact from an evil archaeologist first.'

In [ ]:
resultado = preprocessamento(texto_teste)
resultado

'movie norm north king size adventure richard finn tim maltby alan marriott andrew toth brian dobson cole howard jennifer cameron jonathan holmes lee tockar lisa durupt maya kay michael dobson united states india south korea china 2019 child family movie comedy plan awesome wedding -PRON- grandfather polar bear king steal artifact evil archaeologist'

In [ ]:
lista_sentencas_preprocessada = [preprocessamento(sentenca) for sentenca in lista_sentencas]

In [ ]:
lista_sentencas_preprocessada[0]

'movie norm north king size adventure richard finn tim maltby alan marriott andrew toth brian dobson cole howard jennifer cameron jonathan holmes lee tockar lisa durupt maya kay michael dobson united states india south korea china 2019 child family movie comedy plan awesome wedding -PRON- grandfather polar bear king steal artifact evil archaeologist'

In [ ]:
for _ in range(5):
  i = random.randint(0, len(lista_sentencas) - 1)
  print(lista_sentencas[i])
  print(lista_sentencas_preprocessada[i])
  print('-----')

movie candyman bernard rose virginia madsen, tony todd, xander berkeley, kasi lemmons, vanessa williams, dejuan guy, michael culkin, stanley desantis united states, united kingdom 1992 cult movies, horror movies grad student helen lyle unintentionally summons the candyman, a hook-handed creature made flesh by other people's belief in him.
movie candyman bernard rise virginia madsen tony todd xander berkeley kasi lemmon vanessa williams dejuan guy michael culkin stanley desantis united states united kingdom 1992 cult movie horror movie grad student helen lyle unintentionally summon candyman hook handed creature flesh people belief -PRON-
-----
tv show strongland    2018 docuseries, international tv shows from spain's countryside to scotland's stony terrain, this collection of uplifting documentaries explores the cultural roots of great strongman traditions.
tv strongland 2018 docuserie international tv spain countryside scotland stony terrain collection uplifting documentary explore cul

### **03. Frases de boas-vindas**

In [ ]:
textos_boas_vindas_entrada = ('hey', 'hi', 'hello', 'sup', 'what')
textos_boas_vindas_respostas = ('hey', 'hi', 'hello', 'whats up?', 'welcome!', 'hey, how are you?')

def responder_saudacao(texto):
  if len(texto.split()) > 6:
    return

  texto_lemmas = [token.lower_ for token in pln(texto) if token.lemma_ not in string.punctuation]
  for lemma in texto_lemmas:
    if lemma in textos_boas_vindas_entrada:
      return random.choice(textos_boas_vindas_respostas)  

In [ ]:
responder_saudacao("hey, what's up")

'hello'

### **04. Frases de despedida**

In [ ]:
textos_saida_entrada = ('quit','exit','goodbye', 'bye')
textos_saida_respostas = ('see ya', 'goodbye', 'see you later', 'see you soon')

def quit_sentence(texto):
  if len(texto.split()) > 6:
    return

  texto_lemmas = [token.lower_ for token in pln(texto) if token.lemma_ not in string.punctuation]
  for lemma in texto_lemmas:
    if lemma in textos_saida_entrada:
      return random.choice(textos_saida_respostas)  

In [ ]:
quit_sentence("bye bye")

'see you soon'

### **05. Metodologia**

No pré-processamento da base de dados nós separamos, para cada entrada dela, alguns atributos, como Título, Ano e Descrição, e juntamos em apenas uma sentença. O método de vetorização que utilizaremos não leva em consideração a ordem das palavras e nem a relação entre elas, por isso essa nova sentença não precisa fazer muito sentido e nós só juntamos os atributos de interesse com um espaço em branco. 

Na conversação com o Chatbot, o usuário irá tentar descrever algumas coisas que ele lembra do filme, como personagens, elenco ou descrição, e nós calcularemos quais entradas da base mais tem similaridade com a sentença do usuário, e retornaremos pra ele uma lista de n filmes e/ou séries que possam ser a que ele tentou descrever. 

Iremos utilizar uma vetorização TF-IDF para codificar as sentenças e podermos calcular a proximidade entre elas.  A função de distância que utilizaremos será a [Similaridade por Cosseno](https://en.wikipedia.org/wiki/Cosine_similarity). 

Obs: iremos utilizar os módulos da biblioteca [scikit-learn](http://scikit-learn.org/) para fazer os cálculos, vetorizações e etc.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

### **06: Função para o chatbot responder ao usuário**

In [ ]:
def responder(texto_usuario, max_results):
  lista_sentencas_preprocessada.append(preprocessamento(texto_usuario))

  tfidf = TfidfVectorizer()
  palavras_vetorizadas = tfidf.fit_transform(lista_sentencas_preprocessada)

  lista_sentencas_preprocessada.remove(preprocessamento(texto_usuario))

  similaridade = cosine_similarity(palavras_vetorizadas[-1], palavras_vetorizadas)

  vetor_similar = similaridade.flatten()
  vetor_similar.sort()

  vetores_encontrados = [vetor_similar[-2-i] for i in range(max_results)]
  n_vetores_positivos = len([v for v in vetores_encontrados if v > 0])

  if (n_vetores_positivos > 0):
    indices_ordenados = similaridade.argsort()[0]
    indices_sentencas = [indices_ordenados[-2-i] for i in range(n_vetores_positivos)]
    resposta_chatbot = data.iloc[indices_sentencas, :].loc[:, interest_columns]
  else:
    resposta_chatbot = "Sorry, I didn't understand that!"  

  return resposta_chatbot

### **07. Conversação**

In [ ]:
from IPython.display import display, clear_output

pd.set_option('display.max_colwidth', None)

In [ ]:
bot_name = "Netflickerson"

bot_description = """
Hi, I am {} and I will try to guess movies and tv shows from Netflix.
You can provide me with a quick description, cast members, years and show type (MOVIE or TV SHOW).
I'll try my best to be helpful!

================================
""".format(bot_name)

print(bot_description)

while (True):
  print("Type your sentence below:")
  texto_usuario = input()
  texto_usuario = texto_usuario.lower()
  if (quit_sentence(texto_usuario) != None):
    clear_output(wait=False)
    print('{}: {}!'.format(bot_name, quit_sentence(texto_usuario)))
    break

  if (responder_saudacao(texto_usuario) != None):
    print('{}: {}\n'.format(bot_name, responder_saudacao(texto_usuario)))
    continue

  clear_output(wait=True)
  print(bot_description)

  print('{}: '.format(bot_name))
  print("Your sentence was: {}\n".format(texto_usuario))
  print("My results are:")
  display(responder(texto_usuario, max_results=3))
  print("\n")

Netflickerson: see ya!


### **08. Conclusão**

Ficamos satisfeitos com o resultado desse projeto, pois o bot pode ser uma ferramenta de busca utilizada para tentar identificar nomes de shows dos quais o usuário não se lembra, ou então para identificar shows em que algum ator ou atriz de sua preferência aparece, entre outras aplicações.

Não ficou uma ferramenta de busca tão rebuscada, pois algumas sentenças simples como "a movie with \[...\]" não filtram apenas filmes, pois é uma palavra que repete quase metade da base de dados, então o algoritmo não da muita importância pra ela por causa da vetorização TF-IDF.

Apesar disso, buscas como "Madagascar" ou "Transformers" filtram bem melhor os resultados, pois são palavras que aparecem em poucos shows.

Nós tentamos deixar a conversação amigável pro usuário e formatamos a saída para que fosse bem intuitiva. Acreditamos que o resultado ficou uma experiência agradável.